In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Load the dataset
file_path = 'train.csv'  # Replace with your CSV file path
data = pd.read_csv(file_path)

In [3]:
# Applying logarithmic transformations
data['full_sq_log'] = np.log1p(data['full_sq'])
data['leisure_count_500_log'] = np.log1p(data['leisure_count_500'])
data['life_sq_log'] = np.log1p(data['life_sq'])
data['cafe_count_500_price_high_log'] = np.log1p(data['cafe_count_500_price_high'])

In [4]:
# Add log-transformed features to the list of key features
key_features = ['full_sq', 'life_sq', 'floor', 'leisure_count_500', 'cafe_count_1000_price_high',
                'mosque_count_500', 'mosque_count_1000', 'cafe_count_500_price_high',
                'cafe_count_1000_price_high', 'cafe_count_500_price_4000', 'culture_objects_top_25_raion', 
                'leisure_count_1000', 'trc_sqm_500', 'church_count_500', 'cafe_count_1000_price_1500', 
                'mosque_count_1500', 'cafe_count_500', 'big_church_count_500',
                'public_transport_station_min_walk', 'metro_min_walk', 'kindergarten_km', 'preschool_km',
                'full_sq_log', 'life_sq_log', 'leisure_count_500_log', 'cafe_count_500_price_high_log']  # Add log-transformed features
data = data[key_features + ['price_doc']]

In [5]:
# Handling any potential infinities or NaNs
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(0, inplace=True)

# Splitting the data into features (X) and target variable (y)
X = data.drop('price_doc', axis=1)
y = data['price_doc']

# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling using Robust Scaling
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [6]:
# Define the learning rate
learning_rate = 0.001  # You can change this value as needed
batch_size = 32 # Adjust the batch size

In [7]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.4),  # Increased dropout rate
    Dense(128, activation='relu'),
    Dropout(0.3),  # Added dropout layer
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Output layer for regression
])


In [8]:
# Compiling the Model with a custom learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=[RootMeanSquaredError(name='rmse')])


In [9]:
# Implementing Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True)

In [10]:
# Training the Model with the specified batch size
history = model.fit(X_train_scaled, y_train, batch_size=32, epochs=150, validation_data=(X_val_scaled, y_val), callbacks=[early_stopping])

Epoch 1/150
4538/4538 [==============================] - 9s 2ms/step - loss: 234781367336960.0000 - rmse: 15322577.0000 - val_loss: 195599353774080.0000 - val_rmse: 13985684.0000
Epoch 2/150
4538/4538 [==============================] - 8s 2ms/step - loss: 192828781101056.0000 - rmse: 13886280.0000 - val_loss: 183584300204032.0000 - val_rmse: 13549328.0000
Epoch 3/150
4538/4538 [==============================] - 8s 2ms/step - loss: 189024564150272.0000 - rmse: 13748620.0000 - val_loss: 181197774782464.0000 - val_rmse: 13460972.0000
Epoch 4/150
4538/4538 [==============================] - 8s 2ms/step - loss: 186546317688832.0000 - rmse: 13658196.0000 - val_loss: 179463195197440.0000 - val_rmse: 13396387.0000
Epoch 5/150
4538/4538 [==============================] - 8s 2ms/step - loss: 185106195021824.0000 - rmse: 13605374.0000 - val_loss: 177948128706560.0000 - val_rmse: 13339720.0000
Epoch 6/150
4538/4538 [==============================] - 7s 2ms/step - loss: 184315015069696.0000 - rmse:

In [ ]:
# Model Summary
model.summary()

In [ ]:
# Evaluating the model
results = model.evaluate(X_val_scaled, y_val)
print(f"Validation RMSE: {results[1]}")

In [ ]:
# Load the test dataset
test_file_path = 'test.csv'  # Replace with your test CSV file path
test_data = pd.read_csv(test_file_path)

# Applying the same logarithmic transformations as in the training data
test_data['full_sq_log'] = np.log1p(test_data['full_sq'])
test_data['leisure_count_500_log'] = np.log1p(test_data['leisure_count_500'])
test_data['life_sq_log'] = np.log1p(test_data['life_sq'])
test_data['cafe_count_500_price_high_log'] = np.log1p(test_data['cafe_count_500_price_high'])

# Handling any potential infinities or NaNs
test_data.replace([np.inf, -np.inf], np.nan, inplace=True)
test_data.fillna(0, inplace=True)

# Selecting the same features as used in the training data
X_test = test_data[['full_sq', 'life_sq', 'floor', 'leisure_count_500', 'cafe_count_1000_price_high',
                    'mosque_count_500', 'mosque_count_1000', 'cafe_count_500_price_high',
                    'cafe_count_1000_price_high', 'cafe_count_500_price_4000', 'culture_objects_top_25_raion', 
                    'leisure_count_1000', 'trc_sqm_500', 'church_count_500', 'cafe_count_1000_price_1500', 
                    'mosque_count_1500', 'cafe_count_500', 'big_church_count_500',
                    'public_transport_station_min_walk', 'metro_min_walk', 'kindergarten_km', 'preschool_km',
                    'full_sq_log', 'life_sq_log', 'leisure_count_500_log', 'cafe_count_500_price_high_log']]


# Scale the test data using the same scaler as the training data
# Assuming the scaler is already fitted on the training data
X_test_scaled = scaler.transform(X_test)

# Make predictions using the trained model
# Assuming 'model' is your trained model instance from the same notebook session
predictions = model.predict(X_test_scaled)

# Prepare submission
submission = pd.DataFrame({
    'row ID': test_data['row ID'],  # Replace 'ID' with the identifier column of your test dataset
    'price_doc': predictions.flatten()
})
submission.to_csv('predictions.csv', index=False)
